<a href="https://colab.research.google.com/github/konstantin224/Diplom/blob/main/%D0%98%D0%B7%D0%B2%D0%BB%D0%B5%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B8%D0%BC%D0%B5%D0%BD%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D1%81%D1%83%D1%89%D0%BD%D0%BE%D1%81%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import re


torch.manual_seed(42)





In [ ]:
pip install spacy[transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 35.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [ ]:
import json

with open('NER_data.json') as f:
    data = json.load(f)




In [ ]:
data

[{'Unnamed: 0': 2,
  'message_text': '#Вакансия #Ростов-на-Дону #Fullstack #PHP #Laravel #Vue #Nuxt #fulltime #офис #гибрид\nВсем привет!\nМы - Digital-агентство UMAX, которое ищет Full-stack разработчика со знанием PHP, если ты хорош во Vue.js и PHP - откликайся, твой опыт будет полезен и у нас есть что тебе предложить. \n\nКомпания помогает с переездом и выплачивает релокационный бонус. Для иногородних сотрудников предоставляется квартира рядом с офисом.\n\nОбязанности:\n➡ Участие в разработке и развитии сложных и интересных веб-проектов.\n➡ Разработка решений на различных платформах (выбор оставляем за тобой).\nПоддержка существующих решений.\n➡ Общение с коллегами разработчиками для поиска оптимального решения поставленной задачи.\n➡ Разработка решений на различных платформах (выбор оставляем за тобой).\n\nТребуется:\n✅ Уверенные знания PHP, желателен опыт работы в студии/агентстве\n✅ Опыт работы с Laravel \n✅ Опыт работы с MySQL или иными реляционными СУБД.\n✅ Уверенные знания HTM

In [ ]:
training_data = {'classes' : ['REQUIREMENTS', "RESPONSIBILITIES", "OFFER"], 'annotations' : []}
for example in data:
  temp_dict = {}

  text = example['message_text']


  temp_dict['text'] = example['message_text']


  temp_dict['entities'] = []
  for annotation in example['label']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['labels'][0].upper()
    temp_dict['entities'].append((start, end, label))
  training_data['annotations'].append(temp_dict)

In [ ]:
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    # invalid_span_tokens = re.compile(r'\s')
    # invalid_span_tokens_1 =  re.compile(r'\n')
    # invalid_span_tokens_2 =  re.compile(r'/^ /')

    cleaned_data = []
    for sent in data["annotations"]:
        temp_dict = {}
        temp_dict['text'] = sent['text']
        temp_dict['entities'] = []

        text = sent["text"]
        entities = sent['entities']

        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and (text[valid_start].startswith(' ') or text[valid_start].startswith('\n') or text[valid_start].startswith('\xa0') or text[valid_start].startswith('\t')):
                valid_start += 1
                print(text)
            while valid_end > 1 and (text[valid_end - 1].endswith(" ") or text[valid_end - 1].endswith("\n") or text[valid_end - 1].endswith("\xa0") or text[valid_end - 1].endswith("\t")):
                valid_end -= 1
                print(text)
            temp_dict['entities'].append((valid_start, valid_end, label))
        cleaned_data.append(temp_dict)

    return cleaned_data

In [ ]:
training_data = trim_entity_spans(training_data)

#вакансия #iOS #remote #удалёнка #swift #objective-c

Вакансия: разработчик на Swift 
Формат работы: удалёнка
Занятость: полная / частичная
Зарплата: от 30.000 до 120.000

Описание вакансии:
В небольшую команду для работы над уже существующими проектами ищем разработчика. 
Наш будущий сотрудник человек, который ищет решения, а не проблемы. Человек, нацеленный на рост и развитие вместе с командой. 

Рассматриваем как новичков, так и опытных разработчиков.
 
Что мы предлагаем:
- Достойная заработная плата
- Работа в растущей компании над интересными проектами
- Дружный коллектив
- Четко выстроенные рабочие процессы без бюрократии

Кто мы:
Компания White & Fluffy. 
Мы занимаемся разработкой утилит под iOS. Портфель проектов разнообразный: от гаданий до видео редакторов. 

Контакты:
@babitskiy - пожалуйста, сразу присылайте своё резюме.
Вакансия  — Удалённо 

🕹 Должность 

Сторисмейкер

🕹 Работодатель

Блогер https://instagram.com/ah__anna?igshid=9x2uhfwiln0p

🕹  Список обязанностей:

Анал

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("ru") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
! python -m spacy init fill-config base_config.cfg config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

100%|██████████| 433/433 [00:01<00:00, 357.83it/s]


In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0


ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 265kB/s]
config.json: 100% 625/625 [00:00<00:00, 3.24MB/s]
vocab.txt: 100% 872k/872k [00:00<00:00, 3.57MB/s]
tokenizer.json: 100% 1.72M/1.72M [00:00<00:00, 5.90MB/s]
model.safetensors: 100% 672M/672M [00:02<00:00, 252MB/s]
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipel

In [ ]:
! python -m spacy debug data ./config.cfg --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(

============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: ru
Training pipeline: transformer, ner
374 training docs
374 evaluation docs
⚠ 373 training examples also in evaluation data
⚠ Low number of examples to train a new pipeline (374)

============================== Vocab & Vectors ==============================
ℹ 87349 total word(s

In [ ]:
import spacy_transformers

In [ ]:
nlp_ner = spacy.load("model-best")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
txt = "​Администратор-экскурсовод | Музей Эпос Калевала, Сортавала | от 25000 рублей\
Ищут человека, неравнодушного к северной красоте Карелии, который готов уехать на край света и с упоением рассказывать о тайнах местных минералов\
Какие требования:\
- Неподдельный интерес к природе сурового Севера;\
- Базовые познания в минералогии и желание приумножать свои знания;\
- Умение рассказывать так, чтобы все ждали продолжения, затаив дыхание;\
- Хорошо поставленная грамотная речь;\
- Опыт работы в туристической сфере будет преимуществом.\
Какие обязанности:\
- Знакомство гостей музея с минералогическими богатствами Карельского края и местной арт-галереей;\
- Проведение индивидуальных и групповых экскурсий;\
- Организация мастер-классов и обучающих мероприятий;\
- Продажа уникальных сувениров из лавки при музее.\
\
Какие условия:\
- Обучение и помощь в освоении со стороны музея;\
- Испытательный срок 2 месяца, повышение заработной платы после успешного прохождения;\
- Гибкий график с плавающими выходными;\
- Работа в одном из самых живописных краев России.\
\
Откликнуться на вакансию можно, отправив резюме на почту best.sortavala@gmail.com"

In [ ]:
doc = nlp_ner(txt)

colors = {"RESPONSIBILITIES": "#F67DE3", "REQUIREMENTS": "#7DF6D9", "OFFER":"#f20505"}
options = {"colors": colors}

In [ ]:
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
! spacy evaluate model-best/ training_data.spacy --gpu-id 0

ℹ Using GPU: 0
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(

================================== Results ==================================

TOK     100.00
NER P   94.10 
NER R   91.57 
NER F   92.82 
SPEED   5145  


=============================== NER (per type) ===============================

                       P       R       F
RESPONSIBILITIES   94.05   93.05   93.55
REQUIREMENTS       93.73   90.62   92.15
OFFER              94.68   91.35   92.99

